In [235]:
#-----------------------------------------------------------------------
# cont_flow_analyzer_given_variables.py
# Author: Rebecca Barber
# 
#-----------------------------------------------------------------------

import scipy.stats as st
from statistics import *
import matplotlib.pyplot as plt
from sys import argv
import numpy as np
from math import *
import pandas as pd
from plotnine import *
from random import * 
from sympy import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# The next block is just like a sandbox

In [263]:
q1 = beta
q2 = (alpha*beta) / (2-alpha)
# q2 = (2*alpha - alpha*beta - 2 + 2 * beta) / alpha

dq1_da = q1.diff(alpha)
dq2_da = q2.diff(alpha)
dq1_db = q1.diff(beta)
dq2_db = q2.diff(beta)
    
    
    # find function for h(alpha, beta) = PDF wrt alpha and beta
h = dq1_da * dq2_db - dq2_da * dq1_db

    # need to make sure that h is positive. to do so, evaluate h at (alpha, beta) = (1/2, 1/2)
test = h.subs(alpha, .5).subs(beta, .5)
if test <= 0: h = -h


    # define the derivative of lambda = -h
dlambda_db = -h

    # find lambda (minus the constant) given the derivative
lambda_minus_c = integrate(dlambda_db, beta)

    # find value for the constant
    # let's assume for now that alpha and beta are both drawn uniformly from 0 to 1

    # we want lambda(beta = 1) to be weakly negative 
    # really, we want lambda(beta = 1) = 0 because we want c to be as big as possible.
    # so lambda_minus_c(beta = 1) + c = 0 ==> c = -lambda_minus_c(beta = 1).

    # note that we also need lambda(beta = 0) to be weakly positive, but that gives us a lower bound
    # on c, which we don't really care about 
c = - lambda_minus_c.subs(beta, 1)

lamb = lambda_minus_c + c


    # now, we can compute the virtual values
vv1_part1 = 1/(1-q1)
vv1_part2 = (dq1_db * lamb)/(h * (1-q1)**2)
vv1 = simplify(vv1_part1 - vv1_part2)


vv2_part1 = 1/(1-q2)
vv2_part2 = (dq2_db * lamb)/(h * (1-q2)**2)
vv2 = simplify(vv2_part1 - vv2_part2)

vv1
vv2

-1/(2*beta)

(alpha - 2)*(-alpha*(alpha - 2)**2*(beta**2 - 1) + 2*beta*(alpha**2 - 4*alpha + 4)*(alpha*beta + alpha - 2))/(2*beta*(alpha**2 - 4*alpha + 4)*(alpha*beta + alpha - 2)**2)

# Calculate virtual value functions given quantile functions

In [255]:
def get_vv_functions(q1, q2):
    # take derivatives of alpha, beta with respect to q1 and q2
    dq1_da = q1.diff(alpha)
    dq2_da = q2.diff(alpha)
    dq1_db = q1.diff(beta)
    dq2_db = q2.diff(beta)
    
    print('dq1_da:', dq1_da)
    print('dq2_da:', dq2_da)
    print('dq1_db:', dq1_db)
    print('dq2_db:', dq2_db)
    
    # find function for h(alpha, beta) = PDF wrt alpha and beta
    h = dq1_da * dq2_db - dq2_da * dq1_db

    # need to make sure that h is positive. to do so, evaluate h at (alpha, beta) = (1/2, 1/2)
    test = h.subs(alpha, .5).subs(beta, .5)
    if test <= 0: h = -h
    
    print('h:', simplify(h))

    # define the derivative of lambda = -h
    dlambda_db = -h

    # find lambda (minus the constant) given the derivative
    lambda_minus_c = integrate(dlambda_db, beta)

    # find value for the constant
    # let's assume for now that alpha and beta are both drawn uniformly from 0 to 1

    # we want lambda(beta = 1) to be weakly negative 
    # really, we want lambda(beta = 1) = 0 because we want c to be as big as possible.
    # so lambda_minus_c(beta = 1) + c = 0 ==> c = -lambda_minus_c(beta = 1).

    # note that we also need lambda(beta = 0) to be weakly positive, but that gives us a lower bound
    # on c, which we don't really care about 
    c = - lambda_minus_c.subs(beta, 1)
    print('c:', simplify(c))
    lamb = lambda_minus_c + c
    print('lambda:', simplify(lamb))

    # now, we can compute the virtual values
    vv1_part1 = 1/(1-q1)
    vv1_part2 = (dq1_db * lamb)/(h * (1-q1)**2)
    vv1 = simplify(vv1_part1 - vv1_part2)
    print('vv1:', vv1)

    vv2_part1 = 1/(1-q2)
    vv2_part2 = (dq2_db * lamb)/(h * (1-q2)**2)
    vv2 = simplify(vv2_part1 - vv2_part2)
    print('vv2:', vv2)
    # there is a typo for vv2 in thesis :( for (0,0) flow

    return vv1, vv2

In [258]:
# define alpha and beta
alpha, beta = symbols('alpha beta')

q1 = beta
# q2 = (alpha*beta) / (2-alpha)
q2 = (2*alpha - alpha*beta - 2 + 2 * beta) / alpha
 
get_vv_functions(q1, q2)

dq1_da: 0
dq2_da: (2 - beta)/alpha - (-alpha*beta + 2*alpha + 2*beta - 2)/alpha**2
dq1_db: 1
dq2_db: (2 - alpha)/alpha
h: (2 - 2*beta)/alpha**2
c: alpha**(-2)
lambda: (beta**2 - 2*beta + 1)/alpha**2
vv1: -1/(2*beta - 2)
vv2: alpha/(2*(alpha*beta - alpha - 2*beta + 2))


(-1/(2*beta - 2), alpha/(2*(alpha*beta - alpha - 2*beta + 2)))

# ER draws


In [136]:
def draw_alpha_beta():
    alpha = random()
    beta = random()
    return alpha, beta

# Flow Simulation


In [229]:
# simulates flow given n bidders and m items
# essentially just arranging the n*m values and using
# the given mechanism to "star" values
def calc_vvs(n, vv1, vv2):
    # calc vvs for each bidder
    all_vv1s = []
    all_vv2s = []
    
    for i in range(n):
        alpha, beta = draw_alpha_beta()
        
        # substitution is too slow I think
        vv1_val = vv1(alpha, beta)
        vv2_val = vv2(alpha, beta)
#         vv1_val = vv1.subs(alpha, x).subs(beta, y)
#         vv2_val = vv2.subs(alpha, x).subs(beta, y)

        all_vv1s.append(vv1_val)
        all_vv2s.append(vv2_val)
    
    max_vv1 = max(max(all_vv1s),0)
    max_vv2 = max(max(all_vv2s),0)
    return max_vv1 + max_vv2

# Variable Updates

In [230]:
num_trials = 10000 # 100000
min_bidders = 100
max_bidders = 500
bidder_step = 20

num_bidders = []
for i in range(min_bidders, max_bidders+1, bidder_step):
    num_bidders.append(i)
print(num_bidders)

[100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500]


In [231]:
# define alpha and beta
alpha, beta = symbols('alpha beta')

# define the functions for alpha and beta

# # from the (0,0) flow
# q1 = beta
# q2 = alpha*beta

# from the pure diag flow
# q1 = alpha + beta - alpha*beta
# q2 = beta * (1-alpha)

# from (1,1) flow
q1 = alpha+beta * (1-alpha)
q2 = beta

# Main


In [232]:
avg_revs = []
vv1, vv2 = get_vv_functions(q1, q2)
vv1_numpy = lambdify([alpha,beta], vv1, "numpy")
vv2_numpy = lambdify([alpha,beta], vv2, "numpy")

print('number of bidders:')
for n in range(min_bidders, max_bidders+1, bidder_step):
    print(n, end = "")

    # run num_trials for each # of bidders so we can 
    # take the average
    all_revs = []
    for i in range(num_trials):
        max_rev = calc_vvs(n, vv1_numpy, vv2_numpy)
        all_revs.append(max_rev)  
    mean_rev = mean(all_revs)
    avg_revs.append(mean_rev)
    print(':', mean_rev)

# save all of the data
csv_file = './data/cont_flow_analyzer_given_variables_' + str(min_bidders) + 'to' + \
    str(max_bidders) + 'bidders_' + str(bidder_step) + 'step_' + \
    str(num_trials) + 'trials.csv'

df = pd.DataFrame(columns=['num bidders', 'avg adj rev'])
for i in range(len(num_bidders)):
    n = num_bidders[i]
    avg_rev = avg_revs[i]
    df = df.append({'num bidders': n, 'avg adj rev': avg_rev}, ignore_index=True)

df.to_csv(csv_file)

number of bidders:
100: 9488.382662403152
120: 18895.102607888177
140

KeyboardInterrupt: 

In [ ]:
# plot the results
title = 'Fllloooowww'
log_bench = [1.6*np.log(n) for n in num_bidders]
sqrt_bench = [.75*np.sqrt(n) for n in num_bidders]

plt1 = plt.style.use('ggplot')
plt.figure(figsize=(9,5))
with plt.rc_context({'axes.edgecolor':'black', 'xtick.color':'black', 'ytick.color':'black'}):
    plt.plot(num_bidders, log_bench, 'r-',
        num_bidders, sqrt_bench, 'b-',
         num_bidders, avg_revs,'ko', 
         markersize=3, linewidth = 4)
plt.legend(['c1 * log(n)', 'c2 * sqrt(n)', 'Adjusted Revenue'])
plt.title(title, fontsize = 24)
xlab = plt.xlabel('Number of Bidders', fontsize=16)
ylab = plt.ylabel('Adjusted Revenue', fontsize=16)
xlab.set_color('black')
ylab.set_color('black')

In [226]:
# For fitting y = B + A log x, just fit y against (log x)
# For fitting y = B + A sqrt n, just fit y against (sqrt x)
# https://stackoverflow.com/questions/3433486/how-to-do-
# exponential-and-logarithmic-curve-fitting-in-python-i-found-only-poly
log_fit = np.polyfit(np.log(num_bidders), avg_revs, 1, full = True)
sqrt_fit = np.polyfit(np.sqrt(num_bidders), avg_revs, 1, full = True)
print('log fit: \na =', log_fit[0][0], '\nb =', log_fit[0][1], '\nerror =', log_fit[1][0])
print('\nsqrt fit: \na =', sqrt_fit[0][0], '\nb =', sqrt_fit[0][1], '\nerror =', sqrt_fit[1][0])

log fit: 
a = 5.638789501091301 
b = -19.089518807244072 
error = 1.963922360938208

sqrt fit: 
a = 0.7169062838218871 
b = 0.3887196751882327 
error = 0.9579898392862098
